In [22]:
import pandas as pd
import numpy as np
import random
import pickle
import sys
from keras.models import load_model
from keras import backend
import json 

sys.path.append("../../src/models/predict_model")
import predictor
sys.path.append("../../src/data")
import exclusionCriteria

pd.options.mode.chained_assignment = None  # default='warn'


## Get one record for a sample prediction

In [7]:
fileName = 'ED_TOTAL_2009_2009.csv'
with open('../../fileConfig.json') as config_file:    
        fileConfig = json.load(config_file)
fileLocation = fileConfig['dataDirectory'] + fileConfig['processedDirectory'] + fileName
cdc_file = pd.read_csv(fileLocation)        

In [18]:
# records with vital signs = -9 are filtered out for demo purposes
# todo: resolve records with missing data 
reload(exclusionCriteria)
cdc_file = exclusionCriteria.filter_out_records_na(cdc_file)


In [19]:
#getting one record from the input file, as an example
record_index = random.randint(0,len(cdc_file))
sample_record = cdc_file.iloc[[record_index]]
sample_record.head()

,VYEAR,VMONTH,VDAYR,AGE,ARRTIME,WAITTIME,LOV,RESIDNCE,SEX,ETHUN,...,ADMITHOS,OBSHOS,OBSDIS,OTHDISP,ADMIT,HDSTAT,BDATEFL,IMMEDRFL,REGION,MSA
20771,2009,5,2,29,113,12,43,1,1,-9,...,0,0,0,0,-7,-7,0,0,4,1


##  Read models 

In [4]:
with open('../../modelConfig.json') as config_file:    
        modelConfig = json.load(config_file)
        
modelConfig

{u'co_model': u'../../models/cdc_2009_nn_att_text_embedding.H5',
 u'max_text_length': u'../../models/cdc_2009_att_text_max_length.pickle',
 u'rss_model': u'../../models/cdc_rss_2009_nn_att_text_embedding.H5',
 u'tokenizer': u'../../models/cdc_2009_att_text_tokenizer.pickle'}

## Thresholds

In [30]:
fileESIThreshold_location = fileConfig['dataDirectory'] + fileConfig['resultDirectory'] + 'esi_thresholds.json'  
with open(fileESIThreshold_location) as config_file:    
        thresholdsConfig = json.load(config_file)
thresholdsConfig

{u'esi1_threshold': 0.8709694147109985,
 u'esi2_threshold': 0.7043555974960327,
 u'rss_threshold': 0.35}

## Pass data to predictor

In [31]:
json_data = sample_record.to_json(orient='records')

In [41]:
reload(predictor)
att_top = 15
output = predictor.predict(modelConfig,thresholdsConfig, json_data, att_top)


Creating text for embeddings


In [42]:
print(output)

{"co": 0.03636800870299339, "rss": [2.0], "esi": [3], "co_fi": [{"input_value": "1", "featureName": "Age_18_30_range", "attention_weight": 0.02348059043288231}, {"input_value": "1", "featureName": "Reason_Headache", "attention_weight": 0.021987175568938255}, {"input_value": "2: NOT by ambulance", "featureName": "arrival_model", "attention_weight": 0.021039826795458794}, {"input_value": "1", "featureName": "Reason_Nausea_Vomiting", "attention_weight": 0.020192431285977364}, {"input_value": "12100: ", "featureName": "reason1_for_visit", "attention_weight": 0.019270701333880424}, {"input_value": "13050: ", "featureName": "reason2_for_visit", "attention_weight": 0.017268406227231026}, {"input_value": "0", "featureName": "chf_indicator", "attention_weight": 0.016634879633784294}, {"input_value": "1", "featureName": "Reason_Ears_Eyes_Complaint", "attention_weight": 0.01516253873705864}, {"input_value": "0", "featureName": "Age_81_Above", "attention_weight": 0.014577841386198997}, {"input_val